In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import simbench as sb
import warnings

# Get notebook directory and parent (project root)
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent

# Add project directory to Python path
sys.path.append(str(project_dir))

# Import prepare_results from results_prep
from src.results_prep import prepare_results

In [ ]:
# Set the maximum number of rows to display in total
pd.set_option('display.max_rows', 300)

# Set the minimum number of rows to display before truncating
pd.set_option('display.min_rows', 200)

# set the max columns to none
pd.set_option('display.max_columns', None)

In [ ]:
# Define necessary OPEX parameters
maintenance_percentage_trafo = 0.02 # Source: Verteilnetzstudie Hessen
maintenance_percentage_line = 0.01  # Source: Verteilnetzstudie Hessen

equity_rate = 0.0769  # Source: Eigenkapitalzinssatz für Neuinvestitionen inkl. Gewerbesteuer, BNetzA
equity_percentage = 0.4  # Source: Pauschale Eigenkapitalquote,BNetzA
debt_rate = 0.0171  # Source: Kalkulatorischer Fremdkapitalzinssatz 4. Periode, BNetzA

current_HP_percentage = 5.7 # Source: bdew Report "Wie heizt Deutschland"

HP_tariff_discount = 0.6 # $14a EnWG

sales_tax = 0.19

payback_period = 40  # Source: BNetzA, in years
discount_rate = (equity_percentage * equity_rate) + ((1 - equity_percentage) * debt_rate) # WACC calculation

In [ ]:
# Set parameters
params = {
    'notebook_dir': project_dir,
    'payback_period': payback_period,
    'discount_rate': discount_rate,
    'equity_rate': equity_rate,
    'equity_percentage': equity_percentage,
    'debt_rate': debt_rate,
    'maintenance_percentage_trafo': maintenance_percentage_trafo,
    'maintenance_percentage_line': maintenance_percentage_line,
    'hp_tariff_discount': HP_tariff_discount,
    'sales_tax': sales_tax,
}

# Prepare results DataFrame
results_df = prepare_results(**params)

In [ ]:
# Filter data for HV-mixed and HV-urban grids
hv_mixed_data = results_df[results_df['grid_code'] == 'HV-mixed'].copy()
hv_urban_data = results_df[results_df['grid_code'] == 'HV-urban'].copy()

# Calculate reinforcement cost per household
hv_mixed_data['cost_per_household'] = hv_mixed_data['total_reinforcement_cost'] / hv_mixed_data['total_households']
hv_urban_data['cost_per_household'] = hv_urban_data['total_reinforcement_cost'] / hv_urban_data['total_households']

# Calculate mean and standard deviation at each HP adoption level
hv_mixed_summary = hv_mixed_data.groupby('HP_percentage')['cost_per_household'].agg(['mean', 'std']).reset_index()
hv_urban_summary = hv_urban_data.groupby('HP_percentage')['cost_per_household'].agg(['mean', 'std']).reset_index()

# 2: Network tariff increase

In [ ]:
# Define a function to calculate and plot the required network tariff per kWh for each grid_code in HV level
def plot_required_tariff_per_kwh_with_std(data, payback_period, discount_rate):
    # Filter data for HV grid level only
    hv_data = data[data['grid_level'] == 'HV']
    
    # Set up the color palette and labels for different grid codes
    colors_and_labels = [
        {'color': '#003359', 'label': 'Distribution Grid - rural'},
        {'color': '#A2AD00', 'label': 'Distribution Grid - urban'}
    ]
    
    # Create a figure for the combined plot
    plt.figure(figsize=(12, 8))
    
    # Loop through each unique grid_code in HV level
    for i, grid_code in enumerate(hv_data['grid_code'].unique()):
        # Filter data for the current grid_code
        grid_data = hv_data[hv_data['grid_code'] == grid_code]

        # Store the results for plotting
        hp_adoption_levels = []
        required_tariffs_mean = []
        required_tariffs_std = []

        for hp_percentage in sorted(grid_data['HP_percentage'].unique()):
            # Filter data for the current HP adoption level
            hp_data = grid_data[grid_data['HP_percentage'] == hp_percentage]

            # Calculate required tariff per kWh (mean) and standard deviation
            tariff_mean = hp_data['variable_network_tariff_ct_per_kWh'].mean()
            tariff_std = hp_data['variable_network_tariff_ct_per_kWh'].std()

            # Append results for plotting
            hp_adoption_levels.append(hp_percentage)
            required_tariffs_mean.append(tariff_mean)
            required_tariffs_std.append(tariff_std)
        
        # Use color and label from the colors_and_labels dictionary
        color_info = colors_and_labels[i % len(colors_and_labels)]
        
        # Plot stair-step graph for the mean tariff
        plt.step(hp_adoption_levels, required_tariffs_mean, where='mid', color=color_info['color'], label=f"{color_info['label']}: Mean required additional network tariff")
        
        # Add shaded area for standard deviation
        plt.fill_between(hp_adoption_levels,
                         [m - s for m, s in zip(required_tariffs_mean, required_tariffs_std)],
                         [m + s for m, s in zip(required_tariffs_mean, required_tariffs_std)],
                         color=color_info['color'], alpha=0.2, label=f"{color_info['label']}: Simulation results Std")
    
    # Customize the plot
    plt.title('Required Additional Network Tariff per kWh by HP Adoption Level')
    plt.xlim(0, 100)
    plt.ylim(0)
    plt.xlabel('HP Adoption Level (%)')
    plt.ylabel('Required Additional Network Tariff (ct per kWh)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Apply the function to plot required network tariff per kWh for each grid_code in HV level
plot_required_tariff_per_kwh_with_std(results_df, payback_period, discount_rate)

In [ ]:
# Define a function to calculate and store the required network tariff per kWh data for each grid_code in HV level
def calculate_required_tariff_table(data, payback_period, discount_rate):
    # Filter data for HV grid level only
    hv_data = data[data['grid_level'] == 'HV']
    
    # Create an empty list to store results
    results = []
    
    # Loop through each unique grid_code in HV level
    for grid_code in hv_data['grid_code'].unique():
        # Filter data for the current grid_code
        grid_data = hv_data[hv_data['grid_code'] == grid_code]

        for hp_percentage in sorted(grid_data['HP_percentage'].unique()):
            # Filter data for the current HP adoption level
            hp_data = grid_data[grid_data['HP_percentage'] == hp_percentage]

            # Calculate required tariff per kWh using the cumulative pv_factor, multiply to get cents per kWh
            tariff_per_kwh = hp_data['variable_network_tariff_ct_per_kWh'].mean()

            # Append the data to the results list
            results.append({
                'grid_code': grid_code,
                'HP_percentage': hp_percentage,
                'required_tariff_per_kWh': tariff_per_kwh
            })

    # Convert the results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Generate the table with required network tariff per kWh data
required_tariff_table = calculate_required_tariff_table(results_df, payback_period, discount_rate)

# Display the table
required_tariff_table

In [ ]:
# Define the current network tariff in ct/kWh
current_tariff = 11.64

# Filter data for HV level only
hv_data = results_df[results_df['grid_level'] == 'HV']

# Calculate the total number of households (mean over all runs)
total_households = hv_data['total_households'].mean()

# Calculate the average household consumption without HP
average_consumption_HH = hv_data['HH_total_kWh_consumed'].mean() / total_households

# Calculate the average household HP consumption if HP installed
average_consumption_HP = hv_data[hv_data['HP_percentage'] == 100]['HP_total_kWh_consumed'].mean() / total_households

# Prepare an empty list to store the results for both grid codes
structured_table_rows = []

# Define grid names and corresponding codes for looping
grid_names = {
    "HV-urban": "Distribution Grid - urban",
    "HV-mixed": "Distribution Grid - rural"
}

# Loop through each HP adoption level to calculate costs
for hp_percentage in sorted(hv_data['HP_percentage'].unique()):
    # Initialize a dictionary for this row with HP adoption level
    row = {('General', 'HP_percentage'): hp_percentage}
    
    # Loop through each grid code to calculate costs based on grid type
    for grid_code, grid_name in grid_names.items():
        # Filter data for the current grid_code and HP adoption level
        grid_data = hv_data[(hv_data['grid_code'] == grid_code) & (hv_data['HP_percentage'] == hp_percentage)]
        
        # Calculate the mean variable network tariff for this HP level and add it to the base tariff
        extra_tariff = grid_data['variable_network_tariff_ct_per_kWh'].mean()
        total_tariff = current_tariff + extra_tariff
        total_tariff_HP = total_tariff * (1 - HP_tariff_discount)

        # Calculate the costs for households without and with heat pumps
        cost_without_hp = (total_tariff/100) * average_consumption_HH
        cost_with_hp = cost_without_hp + (total_tariff_HP/100) * average_consumption_HP

        # Store base costs only at 0% HP adoption level
        if hp_percentage == 0:
            base_cost_without_hp = cost_without_hp
            base_cost_with_hp = cost_with_hp

        # Calculate cost increases for households without and with HP
        increase_without_hp_euros = cost_without_hp - base_cost_without_hp
        increase_without_hp_percentage = (increase_without_hp_euros / base_cost_without_hp) * 100
        increase_with_hp_euros = cost_with_hp - base_cost_with_hp
        increase_with_hp_percentage = (increase_with_hp_euros / base_cost_with_hp) * 100

        # Add results for this grid code to the row with multi-level keys
        row.update({
            (grid_name, 'Average household consumption (kWh)'): average_consumption_HH,
            (grid_name, 'Average household + HP consumption (kWh)'): average_consumption_HP,
            (grid_name, 'Base tariff (ct/kWh)'): current_tariff,
            (grid_name, 'Extra tariff (ct/kWh)'): extra_tariff,
            (grid_name, 'Cost (HH w/o HP) (€)'): cost_without_hp,
            (grid_name, 'Cost increase (HH w/o HP) (€)'): increase_without_hp_euros,
            (grid_name, 'Cost increase (HH w/o HP) (%)'): increase_without_hp_percentage,
            (grid_name, 'Cost (HH with HP) (€)'): cost_with_hp,
            (grid_name, 'Cost increase (HH with HP) (€)'): increase_with_hp_euros,
            (grid_name, 'Cost increase (HH with HP) (%)'): increase_with_hp_percentage
        })

    # Append the row to the list of results
    structured_table_rows.append(row)

# Convert the structured data to a DataFrame with a multi-level column index
structured_table = pd.DataFrame(structured_table_rows)

# Set the multi-level column index
structured_table.columns = pd.MultiIndex.from_tuples(structured_table.columns)

# Set global display format for float numbers to remove scientific notation
pd.options.display.float_format = '{:.4f}'.format

# Display the structured table
structured_table